In [54]:
!pip install -q pandas kagglehub numpy scipy matplotlib scikit-learn ipykernel jupyter pytest

In [45]:
import kagglehub
import pandas as pd
import os

In [53]:
# Download latest version
path = kagglehub.dataset_download("arshkon/linkedin-job-postings")
df = pd.read_csv(os.path.join(path,"postings.csv"))

In [51]:
df.shape

(123849, 31)

In [52]:
df["location"].value_counts().head()

location
United States    8125
New York, NY     2756
Chicago, IL      1834
Houston, TX      1762
Dallas, TX       1383
Name: count, dtype: int64

In [35]:
yearly_df = df[df["pay_period"] == "YEARLY"].shape

In [36]:
ny_df = df[df["location"] == "New York, NY"]

In [37]:
ny_df["pay_period"].value_counts().head()

pay_period
YEARLY     1201
HOURLY      307
MONTHLY       2
WEEKLY        2
Name: count, dtype: int64

In [21]:
us_df = df[df["location"] == "United States"]

In [20]:
ny_df[

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
49,935210241,Einbinder & Dunn LLP,Transactional Attorney,"Growing, boutique law firm seeks a transaction...",NaN,NaN,"New York, NY",2695283.0,1.0,NaN,...,NaN,1.712866e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,10001.0,36061.0
57,1219205895,NaN,Director of Training,Job Posting: Service and Training DirectorComp...,140000.0,YEARLY,"New York, NY",NaN,13.0,NaN,...,NaN,1.712635e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,130000.0,10001.0,36061.0
101,2989631782,ActOne Group,Administrative Assistant - CONCUR,Global Financial Services firm is seeking an e...,90000.0,YEARLY,"New York, NY",3496915.0,1.0,NaN,...,NaN,1.713564e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,82500.0,10001.0,36061.0
135,3407282046,Elica Electric Co,Motion Graphic Designer and Film Editor,Company DescriptionElica Electric Co is an ele...,NaN,NaN,"New York, NY",10957929.0,60.0,NaN,...,NaN,1.712665e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,10001.0,36061.0
287,3745677818,The Morgan Library & Museum,Manager of Human Resources,The Morgan Library & Museum invites applicatio...,94000.0,YEARLY,"New York, NY",50609.0,11.0,NaN,...,NaN,1.713212e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,87500.0,10001.0,36061.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123583,3906259619,MINISO USA,Human Resources Generalist (NY+TX),About the PositionThe HR Generalist plays a ce...,85000.0,YEARLY,"New York, NY",18219383.0,2.0,NaN,...,NaN,1.713569e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,10001.0,36061.0
123613,3906260067,The Metropolitan Museum of Art,"Assistant Curator, Michael C. Rockefeller Wing",About The Metropolitan Museum Of Art\n\nThe Me...,NaN,NaN,"New York, NY",9800.0,3.0,NaN,...,NaN,1.713568e+12,metmuseum.wd5.myworkdayjobs.com,0,FULL_TIME,NaN,NaN,NaN,10001.0,36061.0
123697,3906261001,CNA Insurance,Underwriting Associate(Middle Market Private E...,You have a clear vision of where your career c...,NaN,NaN,"New York, NY",3246.0,4.0,NaN,...,NaN,1.713568e+12,cna.wd1.myworkdayjobs.com,0,FULL_TIME,NaN,NaN,NaN,10001.0,36061.0
123748,3906261749,Pinterest,Client Account Manager II,About Pinterest:\n\nMillions of people across ...,NaN,NaN,"New York, NY",1124131.0,3.0,NaN,...,NaN,1.713572e+12,www.pinterestcareers.com,0,FULL_TIME,NaN,NaN,NaN,10001.0,36061.0


In [9]:
df["description"]

0         Job descriptionA leading real estate firm in N...
1         At Aspen Therapy and Wellness , we are committ...
2         The National Exemplar is accepting application...
3         Senior Associate Attorney - Elder Law / Trusts...
4         Looking for HVAC service tech with experience ...
                                ...                        
123844    Our Walnut Creek office is currently seeking a...
123845    About Pinterest:\n\nMillions of people across ...
123846    Company Overview\n\nEPS Learning is a leading ...
123847    The Business Development Manager is a 'hunter'...
123848    Marketing Social Media Specialist - $70k – $75...
Name: description, Length: 123849, dtype: object